In [ ]:
from numpy.random import seed
seed(101)
from tensorflow import set_random_seed
set_random_seed(101)

import pandas as pd
import numpy as np


#import tensorflow as tf
#from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers import Adam
from keras import optimizers

import os
import cv2

from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline
IMAGE_SIZE = 96
IMAGE_CHANNELS = 3

SAMPLE_SIZE = 80000 # the number of images we use from each of the two classes

In [ ]:
import os 
os.listdir('../input')

In [ ]:
print(len(os.listdir('../input/train')))
print(len(os.listdir('../input/test')))

In [ ]:
import pandas as pd

In [ ]:
import os
import cv2

from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

IMAGE_SIZE = 96
IMAGE_CHANNELS = 3

SAMPLE_SIZE = 80000 # the number of images we use from each of the two classes
INPUT_DIR = '../input/'

In [ ]:
from glob import glob 
from skimage.io import imread 
# Data setup
training_dir = INPUT_DIR + 'train/'
data_frame = pd.DataFrame({'path': glob(os.path.join(training_dir,'*.tif'))})
data_frame['id'] = data_frame.path.map(lambda x: x.split('/')[3].split('.')[0]) 
labels = pd.read_csv(INPUT_DIR + 'train_labels.csv')
data_frame = data_frame.merge(labels, on = 'id')
negatives = data_frame[data_frame.label == 0].sample(SAMPLE_SIZE)
positives = data_frame[data_frame.label == 1].sample(SAMPLE_SIZE)
data_frame = pd.concat([negatives, positives]).reset_index()
data_frame = data_frame[['path', 'id', 'label']]
data_frame['image'] = data_frame['path'].map(imread)

training_path = '../training'
validation_path = '../validation'

for folder in [training_path, validation_path]:
    for subfolder in ['0', '1']:
        path = os.path.join(folder, subfolder)
        os.makedirs(path, exist_ok=True)

training, validation = train_test_split(data_frame, train_size=0.75, stratify=data_frame['label'])

data_frame.set_index('id', inplace=True)

for images_and_path in [(training, training_path), (validation, validation_path)]:
    images = images_and_path[0]
    path = images_and_path[1]
    for image in images['id'].values:
        file_name = image + '.tif'
        label = str(data_frame.loc[image,'label'])
        destination = os.path.join(path, label, file_name)
        if not os.path.exists(destination):
            source = os.path.join(INPUT_DIR + 'train', file_name)
            shutil.copyfile(source, destination)

In [ ]:
len((os.listdir('../validation/0')))

In [ ]:
train_path = '../training'
valid_path = '../validation'

num_train_samples =120000
num_val_samples = 40000
train_batch_size = 10
val_batch_size = 10


train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

In [ ]:
datagen = ImageDataGenerator(rescale=1.0/255)

train_gen = datagen.flow_from_directory(train_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=train_batch_size,
                                        class_mode='binary')

val_gen = datagen.flow_from_directory(valid_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=val_batch_size,
                                        class_mode='binary')


In [ ]:
model = Sequential()
model.add(Conv2D(128, (5, 5), activation = 'relu', input_shape=(96,96, 3), name='Conv_1'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (5, 5), activation = 'relu', name='Conv_2'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32, (3, 3), activation = 'relu', name='Conv_3'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(16, (3, 3), activation = 'relu', name='Conv_4'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(64, kernel_initializer='normal', activation='relu', name='Dense_1'))
model.add(BatchNormalization(name='batch_2'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid', name='Dense_2'))
sgd = optimizers.SGD(lr=0.009)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.summary()

In [ ]:
model.fit_generator(train_gen,validation_data=val_gen,validation_steps=val_steps,epochs=10)

In [ ]:
scores=model.evaluate_generator(val_gen)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))


In [ ]:
model.save('model_final_1.h5')

In [ ]:

testing_files = glob(os.path.join(INPUT_DIR+'test/','*.tif'))
submission = pd.DataFrame()
for index in range(0, len(testing_files), 5000):
    data_frame = pd.DataFrame({'path': testing_files[index:index+5000]})
    data_frame['id'] = data_frame.path.map(lambda x: x.split('/')[3].split(".")[0])
    data_frame['image'] = data_frame['path'].map(imread)
    images = np.stack(data_frame.image, axis=0)
    predicted_labels = [model.predict(np.expand_dims(image/255.0, axis=0))[0][0] for image in images]
    predictions = np.array(predicted_labels)
    data_frame['label'] = predictions
    submission = pd.concat([submission, data_frame[["id", "label"]]])
submission.to_csv('submission.csv', index=False, header=True)

In [ ]:
#results.shape

In [ ]:
#results.head()